In [1]:
# BENEX
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

#* BUERABIBX
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn import neighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_curve, auc, roc_auc_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [2]:
initial_data = pd.read_csv('K:\\Data\\MachineLearningCSV\\MachineLearningCVE\\total4.csv')
initial_data.head(n=5)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
data_to_use = initial_data.dropna()
data_to_use.shape

(2827876, 78)

In [4]:
X = data_to_use.drop(axis=1, columns=[ 'Label'])
y1 = data_to_use['Label'].values

# SMOTE上采样
#smote = SMOTE()
#X , y1 = smote.fit_resample(X, y1)
# ADASYN上采样
#adasyn = ADASYN()
#X, y1 = adasyn.fit_resample(X, y1)

# 划分训练集和测试集
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=123)

In [5]:
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'bool']).columns


In [6]:
import joblib
t = [('ohe', OneHotEncoder(drop='first'), categorical_cols),
('scale', StandardScaler(), numerical_cols)]

col_trans = ColumnTransformer(transformers=t)
col_trans.fit(X_train)
col_trans.fit(X_test)
joblib.dump(col_trans, 'col_trans.joblib')

col_trans = joblib.load('col_trans.joblib')

In [7]:
X_train_transform = col_trans.transform(X_train)
X_test_transform = col_trans.transform(X_test)

# 归一化和标准化特征
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_transform)
X_test_scaled = scaler.transform(X_test_transform)

In [8]:
pd.unique(y1)
target_trans = LabelEncoder()
#y_encoded = target_trans.fit(y1_train)
y_encoded = target_trans.fit_transform(y1_train)
print(pd.unique(y_encoded))

[ 0  2 10  6  4  3  7 12 11  5  1 14 13  8  9]


In [9]:
y1_train_transform = target_trans.transform(y1_train)
y1_test_transform = target_trans.transform(y1_test)

In [10]:
# 指定GPU设备
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.set_visible_devices(physical_devices[0], 'GPU')

In [12]:
# 创建并训练神经网络模型
num_classes = 15
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # 多分类问题需要使用softmax激活函数

In [13]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y1_train_transform, epochs=10, batch_size=32)

Epoch 1/10
70697/70697 [==============================] - 188s 3ms/step - loss: 0.0611 - accuracy: 0.9754
Epoch 2/10
70697/70697 [==============================] - 191s 3ms/step - loss: 0.0403 - accuracy: 0.9843
Epoch 3/10
70697/70697 [==============================] - 193s 3ms/step - loss: 0.0312 - accuracy: 0.9892
Epoch 4/10
70697/70697 [==============================] - 200s 3ms/step - loss: 0.0303 - accuracy: 0.9900
Epoch 5/10
70697/70697 [==============================] - 189s 3ms/step - loss: 0.0287 - accuracy: 0.9911
Epoch 6/10
70697/70697 [==============================] - 199s 3ms/step - loss: 0.0280 - accuracy: 0.9916
Epoch 7/10
70697/70697 [==============================] - 205s 3ms/step - loss: 0.0264 - accuracy: 0.9921
Epoch 8/10
70697/70697 [==============================] - 201s 3ms/step - loss: 0.0276 - accuracy: 0.9922
Epoch 9/10
70697/70697 [==============================] - 220s 3ms/step - loss: 0.0251 - accuracy: 0.9924
Epoch 10/10
70697/70697 [=====================

In [14]:
# 在测试集上评估模型
loss, accuracy = model.evaluate(X_test_scaled, y1_test_transform)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

17675/17675 [==============================] - 42s 2ms/step - loss: 0.0315 - accuracy: 0.9887
Test loss: 0.0314626470208168
Test accuracy: 0.9886717200279236
